In [216]:
# !pip3 install git+git://github.com/geopandas/geopandas.git
!pip3 install descartes
!pip3 install folium
# !pip3 install aiohttp[speedups]
!pip3 install aiohttp
!pip3 install aiodns

In [245]:
from pathlib import Path
import requests
from typing import Any, Iterable, List, Tuple, Callable
import time
import asyncio
import json

import pandas as pd
import geopandas as gpd 
import matplotlib.pyplot as plt
import folium
import aiohttp

In [45]:
county_fips = '45039'

In [46]:
DATA_PATH = Path('../../../data/preprocessed/property_assessor/fips/') / county_fips

In [47]:
parcel_ids_df = pd.read_csv(DATA_PATH / 'parcel_ids.csv')

In [48]:
parcel_ids = list(parcel_ids_df['ParcelId'])

In [ ]:
for parcel_id 

In [49]:
parcel_id = parcel_ids[0]
parcel_id

'216-00-00-005-000'

In [68]:
parcel_id = '126-03-02-058-000'

In [69]:
query_prefix = 'https://apis.estated.com/v4/property'
api_token = 'qrHWe4CrUfn0fYtMrcMQzMkKBdTxxk' # demo
# api_token = 'kLGMMDd14fSTGpvCUBcbSORr0jutul' # sandbox
# api_token = 'qrHWe4CrUfn0fYtMrcMQzMkKBdTxxk' # production
query_url = f'{query_prefix}?token={api_token}&fips={county_fips}&apn={parcel_id}'

In [70]:
content = requests.get(query_url)

In [ ]:
content.json()

In [182]:
# parcel_ids_df['data'] = [None] * len(parcel_ids)
parcel_ids_df['latitude'] = [None] * len(parcel_ids)
parcel_ids_df['longitude'] = [None] * len(parcel_ids)
parcel_ids_df['tax_assessed_value'] = [None] * len(parcel_ids)
parcel_ids_df['tax_market_value'] = [None] * len(parcel_ids)
parcel_ids_df['estimated_market_value'] = [None] * len(parcel_ids)

In [208]:
parcel_ids_df['standardized_land_use_category'] = [None] * len(parcel_ids)
parcel_ids_df['standardized_land_use_type'] = [None] * len(parcel_ids)
parcel_ids_df['frontage'] = [None] * len(parcel_ids)
parcel_ids_df['depth'] = [None] * len(parcel_ids)
parcel_ids_df['area_sq_ft'] = [None] * len(parcel_ids)
parcel_ids_df['building_count'] = [None] * len(parcel_ids)

In [91]:
parcel_ids_df = parcel_ids_df.set_index('ParcelId')

In [132]:
parcel_ids_df.loc['216-00-00-005-000'] = {'Unnamed: 0': 0, 'data': 1}

In [205]:
parcel_ids_df.keys()

Index(['data', 'latitude', 'longitude', 'area_sq_ft',
       'standardized_land_use_category', 'frontage_ft', 'depth_ft',
       'building_count', 'tax_assessed_value', 'tax_market_value',
       'estimated_market_value'],
      dtype='object')

In [ ]:
parcel_ids_df = parcel_ids_df.drop(columns=['land_use_category'])

In [140]:
# parcel_ids_df.loc['114-01-01-021-000']['tax_value'] = -1

In [142]:
# parcel_ids_df

In [134]:
for parcel_id in parcel_ids[10:100]:
    query_url = f'{query_prefix}?token={api_token}&fips={county_fips}&apn={parcel_id}'
    content = requests.get(query_url)
    parcel_ids_df.loc[parcel_id] = {'data': content.json()}

In [251]:
async def donwload_aio(parcel_ids:Iterable[str], parcel_ids_df)->List[Tuple[str, dict]]:
    async def download(parcel_id: str, parcel_ids_df) -> Tuple[str, dict]:
        # print(f"Start downloading {parcel_id}")
        async with aiohttp.ClientSession() as s:
            url = f'{query_prefix}?token={api_token}&fips={county_fips}&apn={parcel_id}'
            resp = await s.get(url)
            out = parcel_id, await resp.json()
            
            parcel_ids_df.loc[parcel_id] = {'data': out[1]}
            populate_data(parcel_ids_df, parcel_id)
        # print(f"Done downloading {url}")
        return out
        
    return await asyncio.gather(*[download(parcel_id, parcel_ids_df) for parcel_id in parcel_ids])

In [252]:
step_size = 100
for idx in range(100, len(parcel_ids), step_size):
    st = time.time()
    parcels_slice = await donwload_aio(parcel_ids[idx:idx+step_size], parcel_ids_df)
    print(f"Asynchronous exec took {time.time() - st} seconds")

Asynchronous exec took 5.735219478607178 seconds
Asynchronous exec took 2.5654876232147217 seconds
Asynchronous exec took 2.5764217376708984 seconds
Asynchronous exec took 2.175962209701538 seconds
Asynchronous exec took 2.361083745956421 seconds


/home/ubuntu/COVID_data_exploration/env/lib/python3.8/site-packages/aiohttp/client.py:850: RuntimeWarning: coroutine 'donwload_aio' was never awaited
  self._request(hdrs.METH_GET, url,


Asynchronous exec took 2.4669318199157715 seconds
Asynchronous exec took 2.6075148582458496 seconds
Asynchronous exec took 2.4184608459472656 seconds
Asynchronous exec took 2.0120391845703125 seconds
Asynchronous exec took 2.742753505706787 seconds
Asynchronous exec took 1.9178638458251953 seconds
Asynchronous exec took 2.004737377166748 seconds
Asynchronous exec took 2.717466115951538 seconds
Asynchronous exec took 2.116724729537964 seconds
Asynchronous exec took 2.5091378688812256 seconds
Asynchronous exec took 2.5763630867004395 seconds
Asynchronous exec took 2.6828644275665283 seconds
Asynchronous exec took 2.2754669189453125 seconds
Asynchronous exec took 1.964780569076538 seconds
Asynchronous exec took 2.9828245639801025 seconds
Asynchronous exec took 2.0305352210998535 seconds
Asynchronous exec took 2.674669027328491 seconds
Asynchronous exec took 3.0223937034606934 seconds
Asynchronous exec took 2.508216619491577 seconds
Asynchronous exec took 2.613919734954834 seconds
Asynchro

TypeError: argument of type 'NoneType' is not iterable

In [128]:
from pprint import pprint

In [254]:
parcel_ids_df.iloc[:-100]

,data,latitude,longitude,area_sq_ft,standardized_land_use_category,frontage_ft,depth_ft,building_count,tax_assessed_value,tax_market_value,estimated_market_value,standardized_land_use_type,frontage,depth
ParcelId,,,,,,,,,,,,,,
216-00-00-005-000,{'error': {'code': 'ISE01'...,None,None,None,None,None,None,None,None,None,None,None,None,None
215-00-00-074-000,{'data': {'metadata': {'pu...,34.2281,-81.0735,1151726,RESIDENTIAL,None,None,None,8028,311400,None,SINGLE FAMILY RESIDENTIAL,None,None
215-00-00-102-000,{'data': {'metadata': {'pu...,34.2305,-81.0756,76666,RESIDENTIAL,None,None,None,8260,206500,249000,SINGLE FAMILY RESIDENTIAL,None,None
026-04-02-005-000,{'data': {'metadata': {'pu...,34.5642,-80.9145,69696,RESIDENTIAL,None,None,None,3968,None,None,SINGLE FAMILY RESIDENTIAL,None,None
026-04-02-007-000,{'data': {'metadata': {'pu...,34.5642,-80.9145,43560,RESIDENTIAL,None,None,None,5500,None,None,SINGLE FAMILY RESIDENTIAL,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
026-04-01-001-000,None,None,None,None,None,None,None,None,None,None,None,None,None,None
163-00-00-047-000,None,None,None,None,None,None,None,None,None,None,None,None,None,None
111-00-00-046-000,None,None,None,None,None,None,None,None,None,None,None,None,None,None


In [ ]:
parcels_slice

In [247]:
def populate_data(parcel_ids_df, parcel_id):
    query_data = parcel_ids_df.loc[parcel_id]['data']
    
    if 'error' in query_data:
        pass
    elif 'data' in query_data:
        data = query_data['data']
        if 'address' in data:
            address = data['address']
            if address:
                if 'latitude' in address:
                    parcel_ids_df.loc[parcel_id]['latitude'] = address['latitude']
                if 'longitude' in address:
                    parcel_ids_df.loc[parcel_id]['longitude'] = address['longitude']
        
        if 'assessments' in data:
            try:
                parcel_ids_df.loc[parcel_id]['tax_assessed_value'] = data['assessments'][0]['total_value']
            except:
                pass
        
        if 'market_assessments' in data:
            try:
                parcel_ids_df.loc[parcel_id]['tax_market_value'] = data['market_assessments'][0]['total_value']
            except:
                pass
            
        if 'valuation' in data:
            try:
                parcel_ids_df.loc[parcel_id]['estimated_market_value'] = data['valuation']['value']
            except:
                pass
        
        if 'parcel' in data:
            try:
                parcel_ids_df.loc[parcel_id]['frontage_ft'] = data['parcel']['frontage_ft']
            except:
                pass
            
            try:
                parcel_ids_df.loc[parcel_id]['depth_ft'] = data['parcel']['depth_ft']
            except:
                pass
            
            try:
                parcel_ids_df.loc[parcel_id]['area_sq_ft'] = data['parcel']['area_sq_ft']
            except:
                pass
            
            try:
                parcel_ids_df.loc[parcel_id]['standardized_land_use_category'] = data['parcel']['standardized_land_use_category']
            except:
                pass
            
            try:
                parcel_ids_df.loc[parcel_id]['standardized_land_use_type'] = data['parcel']['standardized_land_use_type']
            except:
                pass
            
            try:
                parcel_ids_df.loc[parcel_id]['building_count'] = data['parcel']['building_count']
            except:
                pass

In [248]:
for i, parcel_id in enumerate(parcel_ids[:100]):
    populate_data(parcel_ids_df, parcel_id)
    

In [249]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 30)

parcel_ids_df.iloc[:100]
# list(parcel_ids_df.iloc[:100]['market_value'])

,data,latitude,longitude,area_sq_ft,standardized_land_use_category,frontage_ft,depth_ft,building_count,tax_assessed_value,tax_market_value,estimated_market_value,standardized_land_use_type,frontage,depth
ParcelId,,,,,,,,,,,,,,
216-00-00-005-000,{'error': {'code': 'ISE01'...,None,None,None,None,None,None,None,None,None,None,None,None,None
215-00-00-074-000,{'data': {'metadata': {'pu...,34.2281,-81.0735,1151726,RESIDENTIAL,None,None,None,8028,311400,None,SINGLE FAMILY RESIDENTIAL,None,None
215-00-00-102-000,{'data': {'metadata': {'pu...,34.2305,-81.0756,76666,RESIDENTIAL,None,None,None,8260,206500,249000,SINGLE FAMILY RESIDENTIAL,None,None
026-04-02-005-000,{'data': {'metadata': {'pu...,34.5642,-80.9145,69696,RESIDENTIAL,None,None,None,3968,None,None,SINGLE FAMILY RESIDENTIAL,None,None
026-04-02-007-000,{'data': {'metadata': {'pu...,34.5642,-80.9145,43560,RESIDENTIAL,None,None,None,5500,None,None,SINGLE FAMILY RESIDENTIAL,None,None
026-04-02-009-000,{'data': {'metadata': {'pu...,34.5642,-80.9145,98010,RESIDENTIAL,None,None,None,15000,None,121000,SINGLE FAMILY RESIDENTIAL,None,None
216-00-00-019-000,{'data': {'metadata': {'pu...,None,None,204732,RESIDENTIAL,None,None,None,602,None,None,"TIMBERLAND, FOREST, OR TREES",None,None
216-00-00-016-000,{'data': {'metadata': {'pu...,34.2455,-81.0356,254390,RESIDENTIAL,None,None,None,6018,174800,237000,SINGLE FAMILY RESIDENTIAL,None,None
200-00-00-065-000,{'data': {'metadata': {'pu...,34.3188,-80.9058,492228,RESIDENTIAL,None,None,None,1502,None,None,"TIMBERLAND, FOREST, OR TREES",None,None


In [255]:
parcel_ids_df.to_csv(DATA_PATH / 'property_data_detailed.csv')